In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline


def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    left_m = np.array([], dtype=np.float64)
    left_c = np.array([], dtype=np.float64)
    right_m = np.array([], dtype=np.float64)
    right_c = np.array([], dtype=np.float64)
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            m = ((y2-y1)/(x2-x1)) 
            if m > 0.0:
                left_m = np.append(left_m, m)
                c = y1 - m * x1
                left_c = np.append(left_c, c)
            else:
                right_m = np.append(right_m, m)
                c = y1 - m * x1
                right_c = np.append(right_c, c)
                
    left_ave_m = np.mean(left_m)
    left_ave_c = np.mean(left_c)
    right_ave_m = np.mean(right_m)
    right_ave_c = np.mean(right_c)
    
    y1 = 960
    y2 = 320
    
    x1 = ( y1 -  left_ave_c ) / (left_ave_m)
    x2 = ( y2 -  left_ave_c ) / (left_ave_m)
    x1 = int(x1)
    x2 = int(x2)
    cv2.line(img, (x1, y1), (x2, y2), color, thickness)
   
    y1 = 960
    y2 = 320
    x1 = ( y1 -  right_ave_c ) / (right_ave_m)
    x2 = ( y2 -  right_ave_c ) / (right_ave_m)
    x1 = int(x1)
    x2 = int(x2)
    cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def process_image(image):
    gray = grayscale(image)
    gaussian = gaussian_blur(gray, 3)
    cannyedges = canny(gaussian, 50, 150)
    
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(450, 320), (510, 320), (imshape[1],imshape[0])]], dtype=np.int32) 
    #masked_edges = region_of_interest(cannyedges, vertices)

    tmpimg = region_of_interest(cannyedges, vertices)

    rho = 2
    theta = np.pi/180
    threshold = 50
    min_line_length = 150
    max_line_gap = 180
    tmpimg = hough_lines(tmpimg, rho, theta, threshold, min_line_length, max_line_gap)

    tmpimg = weighted_img(tmpimg, image)
    return tmpimg


white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))


[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:08<00:00, 27.16it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 55.9 s, sys: 536 ms, total: 56.4 s
Wall time: 8.7 s
[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████▉| 681/682 [00:29<00:00, 23.06it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 3min 9s, sys: 1.9 s, total: 3min 11s
Wall time: 29.9 s


In [ ]:
Reflections:
    1. The current algorithm works well when there are straight lines, when the lane markings are very clear,
       when the road is even and clear.
    2. We have taken all straight lines out, averaged out slope and intercpets and arrived at new staight line.
    3. So, it will always show striaght lane markings.
    4. The averaging wont work, when there are other edges in the region of interest.
    5. when there is curvature, it will not work.